In [1]:
import requests
from getpass import getpass
import json

In [2]:
session = requests.Session()
session.auth = ('admin', getpass(prompt="Digite sua senha do ElasticSearch"))

Digite sua senha do ElasticSearch········


In [240]:
def search(uri):
    HEADERS = {
        'Content-Type': 'application/json'
    }
    
    query = '''
    {
      "size": 100, 
      "_source": ["texto", "data_publicacao"], 
      "sort": [
       {
         "data_publicacao": {
           "order": "desc"
         }
       }
     ] 
    }
    '''
    
    
    response = session.get(url=f'{uri}/mapa-fatos/_search', headers=HEADERS, data=query)
    assert response.status_code == 200
    
    return json.loads(response.text)['hits']['hits']

In [241]:
results = search("http://elastic.producao.rancher.tcu.gov.br")

for result in results[:5]:
    print(result['_source']['texto'])
    print('-'* 80)
# responses_elasticsearch = result['responses']
# len(responses_elasticsearch)

Em entrevista à Rádio Eldorado, o diretor do Instituto Butantan, Dimas Covas, afirmou que a China deve responder até esta sexta-feira, 14, se enviará a matéria-prima contratada para permitir a retomada da produção da Coronavac no Brasil. Na quarta-feira, 12, o governo paulista se reuniu com autoridades chinesas para alinhar o cronograma. A logística de entrega do IFA está paralisada desde abril, após novos ataques do governo Bolsonaro ao país.  <br>    <br>  O presidente do Butantan afirmou que o último lote de vacinas será entregue nesta sexta-feira: 'São insumos que já deveriam estar aqui em solo brasileiro, por que nesse momento não temos matéria-prima para continuar a produção'.  <br>    <br>  'Nesse momento, não temos autorização do governo da China para importar as vacinas da China. Estamos com intensas negociações com a embaixada aqui no Brasil e com o governo da China através da embaixada brasileira em Pequim.'  <br>    <br>  Segundo Dimas Covas, os papéis que autorizam a impor

In [5]:
from transformers import BertTokenizerFast, BertConfig, \
    Trainer, TrainingArguments, EvalPrediction, IntervalStrategy

import numpy as np
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    classification_report, performance_measure
from typing import Tuple, List, Dict

from torch import nn
from ner.news_dataset import NewsDataset

from sklearn_crfsuite.metrics import flat_classification_report

import torch
from sklearn.model_selection import train_test_split
from ner.process_dataset import ProcessDataset
from ner.model import BertNER
from transformers import BertTokenizerFast
import transformers

import logging
import pickle
from pathlib import Path

# from transformers import pipeline

In [6]:
unique_tags = sorted(['B-LOC', 'I-LOC', 'B-PUB', 'I-PUB', 'B-PESSOA', 'I-PESSOA', 'B-ORG', 'I-ORG', 'O'])

tag2id = {tag: id_gerado for id_gerado, tag in enumerate(unique_tags)}
id2tag = {id_geraddo: tag for tag, id_geraddo in tag2id.items()}

tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased', model_max_length=512,
                                              do_lower_case=False, cache_dir='D:/cache')

model = BertNER.from_pretrained('models/30_epochs_iobe_consistency',
                                cache_dir='D:/cache', id2label=id2tag, label2id=tag2id)

In [236]:
bcolors = {
    "PESSOA": '\033[94m',
    #"TEMPO": '\033[92m',
    "LOC": '\033[93m',
    "ORG": '\033[36m',
    "PUB": '\033[35m',
    #"LEGISLACAO": '\033[36m',
    "ENDC": '\033[0m',
    "O": ""
}

def print_legenda():
    print('LEGENDA DAS CORES:')
    print(' ' * 2, bcolors['PESSOA'] + 'PESSOA' + bcolors['ENDC'])
    print(' ' * 2, bcolors['LOC'] + 'LOCAL' + bcolors['ENDC'])
    print(' ' * 2, bcolors['ORG'] + 'ORGANIZAÇÃO PRIVADA' + bcolors['ENDC'])
    print(' ' * 2, bcolors['PUB'] + 'ENTIDADE PÚBLICA' + bcolors['ENDC'])
print_legenda()    

LEGENDA DAS CORES:
   PESSOA
   LOCAL
   ORGANIZAÇÃO PRIVADA
   ENTIDADE PÚBLICA


In [373]:
special_tokens = [tokenizer.unk_token, tokenizer.sep_token, tokenizer.pad_token, tokenizer.cls_token, tokenizer.mask_token]

def mask_output_iob2(batch_predictions: np.ndarray, j: int, batch_predictions_tags: List):
    # First token: possible predictions: O, B-*. Others = -100 or
    # if last prediction was E-XXX, then only accepts B-XYZ or O or
    # if last prediction was O, then only accepts B-XYZ or O
    if len(batch_predictions_tags) == 0 or batch_predictions_tags[-1][:1] == 'O':
        batch_predictions[j, [id for id, tag in id2tag.items() if tag[:1] not in ['B', 'O']]] = -100
    # if last prediction was B-XXX, then only accepts I-XXX, O or B-XYZ
    elif batch_predictions_tags[-1][:1] == 'B':
        batch_predictions[j, [id for id, tag in id2tag.items() if tag not in [f'I-{batch_predictions_tags[-1][2:]}',
                                                                              'O'] and tag[:1] != 'B']] = -100
    # if last prediction was I-XXX, then only accepts I-XXX, B or O
    elif batch_predictions_tags[-1][:1] == 'I':
        batch_predictions[j, [id for id, tag in id2tag.items() if tag not in [f'I-{batch_predictions_tags[-1][2:]}']
                              and tag[:1] not in ['B', 'O']]] = -100

    return model.config.id2label[np.argmax(batch_predictions[j])]

def decode_preds(tokens, pred_labels, confidences):
    result = []
    preds_entities = pred_labels.copy()
    for index, (token, pred, confidence) in enumerate(zip(tokens, pred_labels, confidences)):
        if token in special_tokens:
            continue
        
        if pred[:2] in ['B-', 'I-', 'E-', 'S-']:
            preds_entities[index] = pred[2:]
        
        if token[:2] == '##':
            result[-1][0] = result[-1][0].replace(bcolors['ENDC'], token[2:] + bcolors['ENDC'])
        else:
            result.append([bcolors[preds_entities[index]] + token + bcolors['ENDC'], np.round(torch.max(confidence).item(), 2)])

    return result

def align_predictions(predictions):
    preds = np.argmax(predictions, axis=-1)
    seq_len = preds.shape[0]
    preds_masked = []

    for j in range(seq_len):
#             preds_list[i].append(id2tag[np.argmax(predictions[i][j])])
        preds_masked.append(mask_output_iob2(predictions, j, preds_masked))
            
    return preds_masked
                

def predict(text):
    token_indices = tokenizer.encode(text)
    tokens_tensor = torch.LongTensor(token_indices).unsqueeze(0)
    # print(tokens_tensor)
#     print(tokenizer.decode(token_indices))
    # model.to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)
    #     print('outputs', outputs)
        confidences = nn.Softmax(dim=-1)(outputs.logits).squeeze()
#         print(outputs.logits.numpy().squeeze(0).shape)
        pred_labels = align_predictions(outputs.logits.numpy().squeeze(0))

    # print('predicted_indices', predicted_indices)
#     pred_labels = [model.config.id2label[predicted_index.item()] for predicted_index in predicted_indices]
    # print(pred_labels)

    tokens = tokenizer.convert_ids_to_tokens(token_indices)
    assert len(tokens) == len(pred_labels)
    print('\n')
    
    return decode_preds(tokens, pred_labels, confidences)


In [522]:
word_tokens = re.findall(r"[\w']+|[-.,!?;()]", results[67]['_source']['texto'])
print('Total word tokens:', len(word_tokens))
print(word_tokens)

Total word tokens: 715
['Quando', 'o', 'conflito', 'se', 'intensifica', ',', 'a', 'luta', 'política', 'em', 'Israel', 'arrefece', '.', 'Quando', 'o', 'país', 'fica', 'à', 'beira', 'da', 'guerra', ',', 'a', 'oposição', 'se', 'alinha', 'ao', 'governo', '.', 'Mas', 'não', 'desta', 'vez', '.', 'Enquanto', 'o', 'confronto', 'em', 'Gaza', 'provoca', 'destruição', 'e', 'a', 'violência', 'sectária', 'entre', 'árabes', 'e', 'judeus', 'se', 'intensifica', ',', 'um', 'dos', 'opositores', 'de', 'Binyamin', 'Netanyahu', 'culpa', 'o', 'primeiro', '-', 'ministro', 'pela', 'sensação', 'de', 'caos', 'e', 'trabalha', 'para', 'tirá', '-', 'lo', 'do', 'cargo', '.', 'br', 'br', 'Yair', 'Lapid', 'disse', 'que', 'os', 'acontecimentos', 'da', 'última', 'semana', "'não", 'podem', 'servir', 'de', 'desculpa', 'para', 'Netanyahu', 'se', 'manter', 'no', "poder'", '.', "'É", 'por', 'isso', 'que', 'ele', 'deveria', 'ser', 'substituído', 'o', 'mais', 'rapidamente', "possível'", ',', 'escreveu', 'Lapid', ',', 'no', 'F

In [521]:
wordpiece_tokens = tokenizer.convert_ids_to_tokens(tokenizer(results[67]['_source']['texto'], padding=True)['input_ids'])
print('Total WordPiece tokens:', len(wordpiece_tokens))
print(wordpiece_tokens)

Total WordPiece tokens: 909
['[CLS]', 'Quando', 'o', 'conflito', 'se', 'intens', '##ifica', ',', 'a', 'luta', 'política', 'em', 'Israel', 'arre', '##fec', '##e', '.', 'Quando', 'o', 'país', 'fica', 'à', 'beira', 'da', 'guerra', ',', 'a', 'oposição', 'se', 'alinha', 'ao', 'governo', '.', 'Mas', 'não', 'desta', 'vez', '.', 'Enquanto', 'o', 'confronto', 'em', 'Gaza', 'provoca', 'destruição', 'e', 'a', 'violência', 'se', '##ct', '##ária', 'entre', 'árabes', 'e', 'judeus', 'se', 'intens', '##ifica', ',', 'um', 'dos', 'opositores', 'de', 'Bi', '##ny', '##ami', '##n', 'Net', '##any', '##ah', '##u', 'culpa', 'o', 'primeiro', '-', 'ministro', 'pela', 'sensação', 'de', 'caos', 'e', 'trabalha', 'para', 'ti', '##rá', '-', 'lo', 'do', 'cargo', '.', '<', 'b', '##r', '>', '<', 'b', '##r', '>', 'Ya', '##ir', 'La', '##pi', '##d', 'disse', 'que', 'os', 'acontecimentos', 'da', 'última', 'semana', "'", 'não', 'podem', 'servir', 'de', 'descul', '##pa', 'para', 'Net', '##any', '##ah', '##u', 'se', 'manter',

In [570]:
# corrigir os tamanhos, pois está quebrando palavras no meio e não separando tokens
max_size = 512 - tokenizer.num_special_tokens_to_add()
text_index_begin = 0
text_index_limit = max_size
print_legenda()

index = 67
# 10, 14, 20 (grande), 46 (grande e com erros) são lixo
# 17, 27, 29 (sem pontuação nem case), 47-50 são gigantescas (53, 54, 56, 58 - 60, 63, 65)
# 17 fora
for result in results[index:index+1]:
    all_tokens = tokenizer.convert_ids_to_tokens(tokenizer(result['_source']['texto'], padding=True)['input_ids'], skip_special_tokens=True)
    while all_tokens[text_index_begin:text_index_limit]:
        found_subword = all_tokens[text_index_limit-1][:2] == '##'
        while all_tokens[text_index_limit-1][:2] == '##':
            text_index_limit -= 1

        if found_subword: #Remove the first subword with no ## at the beginning
            text_index_limit -= 1
        annotated_text = predict(tokenizer.convert_tokens_to_string(all_tokens[text_index_begin:text_index_limit]))
        for item in annotated_text:
            print(item[0], end=' ')
            
        text_index_begin = min(text_index_limit, len(all_tokens))
        text_index_limit = min(text_index_limit+max_size, len(all_tokens))

# for trecho in 
# text = result['_source']['texto'][:512]
# annotated_text = predict(text)
# for item in annotated_text:
#     print(item[0], end=' ')
# for result in results:
#     pass

LEGENDA DAS CORES:
   PESSOA
   LOCAL
   ORGANIZAÇÃO PRIVADA
   ENTIDADE PÚBLICA


Quando o conflito se intensifica , a luta política em Israel arrefece . Quando o país fica à beira da guerra , a oposição se alinha ao governo . Mas não desta vez . Enquanto o confronto em Gaza provoca destruição e a violência sectária entre árabes e judeus se intensifica , um dos opositores de Binyamin Netanyahu culpa o primeiro - ministro pela sensação de caos e trabalha para tirá - lo do cargo . < br > < br > Yair Lapid disse que os acontecimentos da última semana ' não podem servir de desculpa para Netanyahu se manter no poder ' . ' É por isso que ele deveria ser substituído o mais rapidamente possível ' , escreveu Lapid , no Facebook . < br > < br > A escalada da crise ocorre em um momento crucial para a política israelense . Poucos dias atrás , Netanyahu , que é julgado por corrupção e não conseguiu formar governo após quatro eleições em dois anos , parecia prestes a perder o cargo . Lapid obteve a